In [1]:
import tensorflow as tf

In [ ]:
import os
import sys
import threading

import numpy as np
from PIL import Image

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [ ]:
BASE_DIR = os.path.abspath(os.path.dirname(sys.argv[1]))

In [ ]:
BASE_DIR

In [ ]:
def isImageFile(filePath):
    basename_ext = os.path.splitext(filePath)
    ext= basename_ext[1]
    ext = ext.lower()
    if ext=='.bmp': 
        return True
    if ext=='.png': 
        return True
    if ext=='.jpg': 
        return True
    return False

In [ ]:
def filter_for_images(fileList):
    image_list = []
    for i in range(len(fileList)):
        fileName = fileList[i]
        if isImageFile(fileName):
            image_list.append(fileName)
    return image_list

In [ ]:
def find_all_images(dir_path = './'):
    ls_files = os.listdir(dir_path)
    print("There are %d files found in total."%len(ls_files))
    image_list = filter_for_images(ls_files)
    print("And %d image files are found."%len(image_list))
    for i in range(len(image_list)):
        image_list[i] =os.path.join(dir_path,image_list[i])
    return image_list

In [ ]:
def getBaseNames(fileList):
    fileBaseNames=[]
    for i in range(len(fileList)):
        fileName = fileList[i]
        fileBaseNames.append(os.path.splitext(os.path.basename(fileName))[0])
    return fileBaseNames

In [ ]:
def filter_CamVid_ImagePair(imgList,lblList):
    outImgList=[]
    outLblList=[]
    imgBaseNames = getBaseNames(imgList)
    lblBaseNames = getBaseNames(lblList)
    for i in range(len(imgBaseNames)):
        filename = imgBaseNames[i]
        supposedName = filename+"_L"
        if supposedName in lblBaseNames:
            j=lblBaseNames.index(supposedName) 
            outImgList.append(imgList[i])
            outLblList.append(lblList[j])
    return (outImgList,outLblList)

In [ ]:
imgList = find_all_images("../701_StillsRaw_full")
lblList = find_all_images("../LabeledApproved_full")

In [ ]:
baseNames = getBaseNames(imgList)

In [ ]:
outImgList,outLblList = filter_CamVid_ImagePair(imgList,lblList)

In [ ]:
print(outImgList[0],outLblList[0])

In [ ]:
def pase_Label_File(filePath):
    f = open(filePath,mode='r')
    lines = f.readlines()
    label_items=[]
    for i in range(len(lines)):
        item = lines[i]
        item = item.split()
        r = np.uint8(item[0])
        g = np.uint8(item[1])
        b = np.uint8(item[2])
        name = (item[3])
        print((r,g,b,name))
        label_items.append((r,g,b,name))
    f.close()
    return label_items

In [ ]:
def readLabelTxt(filePath):
    basename_ext = os.path.splitext(filePath)
    ext = basename_ext[1].lower()
    label_items = None
    if ext!='.txt':
        raise ValueError('This is not txt file')
    else:
        label_items = pase_Label_File(filePath)
    return label_items  

In [ ]:
label_items = readLabelTxt('../label_colors.txt')

In [ ]:
def findLabelColorofName(label_items,name):
    for i in range(len(label_items)):
        item = label_items[i]
        if(item[3]==name): return (item[0],item[1],item[2])
    return None

In [ ]:
def findLabelNameofColor(label_items,color):
    for i in range(len(label_items)):
        item = label_items[i]
        if((item[0],item[1],item[2])==color or [item[0],item[1],item[2]]==color): return item[3]
    return None

In [ ]:
findLabelColorofName(label_items,'VegetationMisc')

In [ ]:
findLabelNameofColor(label_items,[0, 64, 64])

# Write TFRecords

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def convertDataToTFRecord(imagePath,tfrecoredPath):
    with tf.python_io.TFRecordWriter(tfrecoredPath) as writer:
            im = Image.open(imagePath)
            fim = np.array(im).astype(np.float32)
            #shape = np.array(np.shape(im)).astype(np.int64)
            shape = np.array(np.shape(im)).astype(np.float32)
            feature={'shape':_bytes_feature(tf.compat.as_bytes(shape.tostring())),
                     'train/image':_bytes_feature(tf.compat.as_bytes(fim.tostring()))}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [ ]:
def convertImageToTFRecords(input_dir_path,output_dir_path):
    print("Input Directory is : %s"%input_dir_path)
    print("Output Directory is : %s"%output_dir_path)
    imgList = find_all_images(input_dir_path)
    convertImageToTFRecord(imgList,output_dir_path)
    print("Conversion Done.")

In [ ]:
convertImageToTFRecords("../test","../test")

In [ ]:
convertImageToTFRecords("../701_StillsRaw_full","../tfr_images")

In [ ]:
convertImageToTFRecords("../LabeledApproved_full","../tft_labels")

# Read TFRecords

In [ ]:
def readDatafromTFRecord(sess,tfrecoredPath):
    feature={'shape':tf.FixedLenFeature([],tf.string),
             'train/image':tf.FixedLenFeature([],tf.string)}
    filename_queue = tf.train.string_input_producer([tfrecoredPath],num_epochs=1)
    reader = tf.TFRecordReader()
    _,serialized_example =reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,features=feature)
    image = tf.decode_raw(features['train/image'],tf.float32)
    shape = tf.cast(tf.decode_raw(features['shape'],tf.float32),tf.int32)
    image = tf.reshape(image,shape)
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    img ,val_shape= sess.run([image,shape])
    print(val_shape)
    img = img.astype(np.uint8)
    plt.imshow(img)
    plt.show()
    # Stop the threads
    coord.request_stop()
    # Wait for threads to stop
    
    coord.join(threads)
    

In [ ]:
with tf.Session() as sess:
    readDatafromTFRecord(sess,'/home/yelyu/Work/Dataset/CamVid/test/0001TP_006690.png.tfrecords')

# Read Image directly

# Test My Module

In [1]:
import imp
import ImageReader
imp.reload(ImageReader)
from ImageReader import BatchDataReader
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib notebook

In [2]:
trainingParams = ImageReader.TrainingParams()
trainingParams.set_batchSize(10)
trainingParams.set_epochs(2)
trainingParams.set_lr(1.0)
trainingParams.set_UseRandomShuffle(True)
dataPairListGetter = ImageReader.getCamVidDataPairsList
reader = BatchDataReader("../701_StillsRaw_full","../LabeledApproved_full",
                         '../label_colors.txt',dataPairListGetter,
                         trainingParams)
reader.begin_epoch()

Initializing Batch Dataset Reader...
Looking for images files...
There are 701 files found in total.
in which, 701 image files are found.
Looking for label files...
There are 701 files found in total.
in which, 701 image files are found.
Filtering for image correct label pairs...
There are 701 image pairs in total
Parsing txt file for labels...
There are 32 classes in total


In [3]:
(batch_img,batch_lbl) = reader.next_batch()

sample 0
sample 1
sample 2
sample 3
sample 4
sample 5
sample 6


KeyboardInterrupt: 

In [ ]:
reader.show_batch_image_pair(batch_img,batch_lbl,np.arange(4,8))

# Construct FCN

In [1]:
import imp
import ImageReader
imp.reload(ImageReader)
from ImageReader import BatchDataReader
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib notebook

In [2]:
FLAGS = tf.flags.FLAGS
tf.flags.DEFINE_integer("batch_size","4","batch size for training")
tf.flags.DEFINE_integer("epoch","200","training epochs")
tf.flags.DEFINE_string("logs_dir", "logs/", "path to logs directory")
tf.flags.DEFINE_string("saver_dir","saver/","path to saver directory");
tf.flags.DEFINE_float("learning_rate", "1e-4", "Learning rate for Adam Optimizer")
tf.flags.DEFINE_string('mode', "train", "Mode train/ test/ visualize")
tf.flags.DEFINE_string("Img_dir", "../701_StillsRaw_full", "path to images")
tf.flags.DEFINE_string("Lbl_dir", "../LabeledApproved_full", "path to labels")
tf.flags.DEFINE_string("Txt_dir",'../label_colors.txt',"label color txt")

In [3]:
def max_pool_2x2(x):
    return tf.nn.max_pool(value=x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def weight_variable(shape,name=None):
    initial = None
    if name is None:
        initial = tf.truncated_normal(shape=shape, stddev=0.1)
    else:
        initial = tf.truncated_normal(shape = shape,name=name, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape,name=None):
    initial = None
    if name is None:
        initial = tf.constant(0.0, shape=shape)
    else:
        initial = tf.constant(0.0,name=name, shape=shape)
    return tf.Variable(initial)

def conv_layer(inputTensor,num_input_channels,num_filters,filter_size
               ,stride_size,padding='SAME',useBias=True):
    shape = [filter_size,filter_size,num_input_channels,num_filters]
    weights = weight_variable(shape)
    conv = tf.nn.conv2d(input=inputTensor,filter=weights,strides=[1,stride_size,stride_size,1],
                padding=padding)
    if useBias:
        biases = bias_variable(shape=[num_filters])
        conv=conv+biases
        return conv,weights,biases
    else:
        return conv,weights

def deconv_layer(inputTensor,outputShape,num_input_channels,num_filters,filter_size
               ,stride_size,padding='SAME',useBias=True):
    shape = [filter_size,filter_size,num_input_channels,num_filters]
    weights = weight_variable(shape)
    
    conv = tf.nn.conv2d_transpose(value=inputTensor,filter=weights,output_shape=outputShape,strides=[1,stride_size,stride_size,1],
                padding=padding)
    if useBias:
        biases = bias_variable(shape=[num_filters])
        conv=conv+biases
        return conv,weights,biases
    else:
        return conv,weights
    

In [4]:
def fcn():
    layers = (
        'conv1','relu1','pool1','norm1','conv1','relu1','pool2','norm2','conv3',
        'relu3','conv4','relu4','conv5','relu5','pool5','fc6','relu6','drop6','fc7',
        'relu7','drop7','score_fr','upscore','score'
    )
    net={}

    _Img = tf.placeholder(dtype=tf.float32)
    _Lbl = tf.placeholder(dtype=tf.int32)
    _keep_probability = tf.placeholder(dtype=tf.float32)
    Lbl = tf.expand_dims(_Lbl,axis=3)
    Img = _Img-128.0/128.0
    #conv1
    input_channels = 3;output_channels = 64
    #with tf.VariableScope("Conv1")
    conv1_1,weight1,bias1 = conv_layer(inputTensor=Img,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=3,
                                      stride_size=1,padding='SAME')
    relu1_1 = tf.nn.relu(conv1_1)
    conv1_2,weight2,bias2 = conv_layer(inputTensor=relu1_1,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu1_2 = tf.nn.relu(conv1_2)
    #pool1
    pool1 = max_pool_2x2(relu1_2)
    #######################################################################################
    #conv2
    input_channels = 64;output_channels = 128
    #with tf.VariableScope("Conv2")
    conv2_1,weight1,bias1 = conv_layer(inputTensor=pool1,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=3,
                                      stride_size=1,padding='SAME')
    relu2_1 = tf.nn.relu(conv2_1)
    conv2_2,weight2,bias2 = conv_layer(inputTensor=relu2_1,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu2_2 = tf.nn.relu(conv2_2)
    #pool2
    pool2 = max_pool_2x2(relu2_2)
    #######################################################################################
    #conv3
    input_channels = 128;output_channels = 256
    #with tf.VariableScope("Conv3")
    conv3_1,weight1,bias1 = conv_layer(inputTensor=pool2,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=3,
                                      stride_size=1,padding='SAME')
    relu3_1 = tf.nn.relu(conv3_1)
    conv3_2,weight2,bias2 = conv_layer(inputTensor=relu3_1,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu3_2 = tf.nn.relu(conv3_2)
    conv3_3,weight3,bias3 = conv_layer(inputTensor=relu3_2,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu3_3 = tf.nn.relu(conv3_3)
    #pool3
    pool3 = max_pool_2x2(relu3_3)
    #######################################################################################
    #conv4
    input_channels = 256;output_channels = 512
    #with tf.VariableScope("Conv4")
    conv4_1,weight1,bias1 = conv_layer(inputTensor=pool3,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=3,
                                      stride_size=1,padding='SAME')
    relu4_1 = tf.nn.relu(conv4_1)
    conv4_2,weight2,bias2 = conv_layer(inputTensor=relu4_1,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu4_2 = tf.nn.relu(conv4_2)
    conv4_3,weight3,bias3 = conv_layer(inputTensor=relu4_2,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu4_3 = tf.nn.relu(conv4_3)
    #pool4
    pool4 = max_pool_2x2(relu4_3)
    #######################################################################################
    #conv5
    input_channels = 512;output_channels = 512
    #with tf.VariableScope("Conv5")
    conv5_1,weight1,bias1 = conv_layer(inputTensor=pool4,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=3,
                                      stride_size=1,padding='SAME')
    relu5_1 = tf.nn.relu(conv5_1)
    conv5_2,weight2,bias2 = conv_layer(inputTensor=relu5_1,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu5_2 = tf.nn.relu(conv5_2)
    conv5_3,weight3,bias3 = conv_layer(inputTensor=relu5_2,num_input_channels=output_channels,
                                     num_filters=output_channels,filter_size=3,
                                     stride_size=1,padding='SAME')
    relu5_3 = tf.nn.relu(conv5_3)
    #pool4
    pool5 = max_pool_2x2(relu5_3)
    #######################################################################################
    #fc6
    input_channels = 512;output_channels = 4096
    #with tf.VariableScope("fc6")
    fc6,weight1,bias1 = conv_layer(inputTensor=pool5,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=7,
                                      stride_size=1,padding='SAME')
    relu6 = tf.nn.relu(fc6)
    #dropout6
    drop6 = tf.nn.dropout(relu6,keep_prob=_keep_probability)
    #######################################################################################
    #fc7
    input_channels = 4096;output_channels = 4096
    #with tf.VariableScope("fc7")
    fc7,weight1,bias1 = conv_layer(inputTensor=drop6,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=1,
                                      stride_size=1,padding='SAME')
    relu7 = tf.nn.relu(fc7)
    #dropout6
    drop7 = tf.nn.dropout(relu7,keep_prob=_keep_probability)
    #######################################################################################
    #score_fr
    input_channels = 4096;output_channels = 32
    score_fr,weight1,bias1 = conv_layer(inputTensor=drop7,num_input_channels=input_channels,
                                      num_filters=output_channels,filter_size=1,
                                      stride_size=1,padding='SAME')
    #######################################################################################
    #upscore2
    score_pool4,weight1,bias1 = conv_layer(inputTensor=pool4,num_input_channels=512,
                                      num_filters=32,filter_size=1,
                                      stride_size=1,padding='SAME')
    upscore2,weight1 = deconv_layer(inputTensor=score_fr,outputShape=tf.shape(score_pool4),
                                    num_input_channels=32,num_filters=32,
                                    filter_size=4,stride_size=2,padding='SAME',useBias=False)
    fuse_pool4 = score_pool4+upscore2
    #######################################################################################
    #upscore_pool4
    score_pool3,weight1,bias1 = conv_layer(inputTensor=pool3,num_input_channels=256,
                                      num_filters=32,filter_size=1,
                                      stride_size=1,padding='SAME')
    upscore_pool4,weight1 = deconv_layer(inputTensor=fuse_pool4,outputShape=tf.shape(score_pool3),
                                    num_input_channels=32,num_filters=32,
                                    filter_size=4,stride_size=2,padding='SAME',useBias=False)
    fuse_pool3 = score_pool3+upscore_pool4
    #######################################################################################
    #upscore8
    d =tf.constant([32])
    shape =tf.concat(axis=0,values=[tf.shape(Lbl)[:-1],d])
    upscore8,weight1 = deconv_layer(inputTensor=fuse_pool3,outputShape=shape,
                                    num_input_channels=32,num_filters=32,
                                    filter_size=16,stride_size=8,padding='SAME',useBias=False)
    #######################################################################################
    #loss
    prediction = tf.argmax(upscore8, axis=3,output_type=tf.int32, name="prediction")
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.expand_dims(prediction,axis=3),Lbl) ,tf.float32))
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=upscore8,
                      labels=tf.squeeze(Lbl,axis=3),name="entropy"))
    tf.summary.scalar("loss", loss)
    summary_op = tf.summary.merge_all()
    #net["shape"] = _InputShape
    net["summary"] = summary_op
    net["Prediction"] = prediction
    net["accuracy"] = accuracy
    net["loss"] = loss
    net["loss"] = loss
    net["Img"] =_Img
    net["Lbl"]=_Lbl
    net["Keep_prob"] = _keep_probability
    net["Out"] = Lbl
    return net

In [25]:
def trainNet(net,flags,reader,sess):
    print("Prepare training...")
    print("Setting up Saver...")
    summary_writer = tf.summary.FileWriter(flags.logs_dir+"/train", sess.graph)
    
    #print("Variables Initialized.")
   
    #saver.restore(sess,flags.saver_dir+"model.ckpt")
    optimizer = tf.train.AdamOptimizer(flags.learning_rate)
    
    train_op = optimizer.minimize(net["loss"])

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(keep_checkpoint_every_n_hours=1)
    #ckpt = tf.train.get_checkpoint_state(flags.saver_dir)
    #if ckpt and ckpt.model_checkpoint_path:
    #    saver.restore(sess, ckpt.model_checkpoint_path)
    #    print("Model restored...")
    total_batch_count=0

    for i in range(flags.epoch):
        reader.begin_epoch()
        batchIdx=0
        while True:
            (batch_img,batch_lbl) = reader.next_batch()
            if batch_img==None:
                break
            feed_dict={net["Img"]:batch_img,net["Lbl"]:batch_lbl,net["Keep_prob"] :0.5}
            vals = sess.run([net["loss"],net["accuracy"],train_op,net["summary"]],feed_dict=feed_dict)
      
 
            batchIdx+=1
            print("epoch:%d, batch:%d, loss:%f,accuracy:%f"%(i,batchIdx,vals[0],vals[1]))
            summary_writer.add_summary(vals[3],total_batch_count)
            total_batch_count+=1
        saver.save(sess,flags.saver_dir+"model.ckpt")

In [26]:
def main(FLAGS):
    FLAGS.batch_size = 2
    FLAGS.epoch=100
    FLAGS.learning_rate=0.05
    trainingParams = ImageReader.TrainingParams()
    trainingParams.set_batchSize(FLAGS.batch_size)
    trainingParams.set_epochs(FLAGS.epoch)
    trainingParams.set_lr(FLAGS.learning_rate)
    trainingParams.set_UseRandomShuffle(True)
    dataPairListGetter = ImageReader.getCamVidDataPairsList
    reader = BatchDataReader(FLAGS.Img_dir,FLAGS.Lbl_dir,FLAGS.Txt_dir,
                             dataPairListGetter,trainingParams)
    #tf.reset_default_graph()
    net = fcn()
    with tf.Session() as sess:
        trainNet(net,FLAGS,reader,sess)
    

In [27]:
tf.reset_default_graph()

In [28]:
main(FLAGS)

Initializing Batch Dataset Reader...
Looking for images files...
There are 701 files found in total.
in which, 701 image files are found.
Looking for label files...
There are 701 files found in total.
in which, 701 image files are found.
Filtering for image correct label pairs...
There are 701 image pairs in total
Parsing txt file for labels...
There are 32 classes in total
Prepare training...
Setting up Saver...
sample 0
sample 1


/home/yelyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


FailedPreconditionError: Attempting to use uninitialized value beta1_power
	 [[Node: beta1_power/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/gpu:0"](beta1_power)]]
	 [[Node: Mean/_25 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1754_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'beta1_power/read', defined at:
  File "/home/yelyu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yelyu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yelyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yelyu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 